In [453]:
#!g1.1
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
data = pd.read_csv('/home/jupyter/mnt/s3/main-data/dataset/train.csv')
data

,Unnamed: 0,sentence,1category,2category,sentiment
0,4754,При этом всегда получал качественные услуги.,Communication,NaN,+
1,4417,"Не вижу, за что хотя бы 2 поставить, сервис на 1!",?,NaN,−
2,3629,"Вот так ""Мой любимый"" банк МКБ меня обманул.",?,NaN,−
3,11640,Отвратительное отношение к клиентам.,Communication,NaN,−
4,5571,"Всегда в любое время дня и ночи помогут, ответ...",Communication,NaN,+
...,...,...,...,...,...
19356,8004,Никогда и ни в коем случае не открывайте счет ...,Communication,NaN,−
19357,18182,ТИ откровенно забили на качество и развивают с...,Quality,NaN,−
19358,744,"Я считаю, это прорыв и лидерство финансовых ус...",?,NaN,+
19359,6220,"Писал мужчина очень доходчиво, не финансовым я...",Communication,NaN,+


In [444]:
#!g1.1
data.columns

Index(['Unnamed: 0', 'sentence', '1category', '2category', 'sentiment'], dtype='object')

In [445]:
#!g1.1
data.drop(columns={'Unnamed: 0', '1category', '2category'})

,sentence,sentiment
0,При этом всегда получал качественные услуги.,+
1,"Не вижу, за что хотя бы 2 поставить, сервис на 1!",−
2,"Вот так ""Мой любимый"" банк МКБ меня обманул.",−
3,Отвратительное отношение к клиентам.,−
4,"Всегда в любое время дня и ночи помогут, ответ...",+
...,...,...
19356,Никогда и ни в коем случае не открывайте счет ...,−
19357,ТИ откровенно забили на качество и развивают с...,−
19358,"Я считаю, это прорыв и лидерство финансовых ус...",+
19359,"Писал мужчина очень доходчиво, не финансовым я...",+


In [446]:
#!g1.1
sentences = data['sentence'].values
y = data['sentiment'].values
y = np.where(y=='−', 0, np.where(y=='+', 1, 2))

In [447]:
#!g1.1
train_size = int(len(sentences) * 0.8)
train_sentences = sentences[:train_size]
train_y = y[:train_size]
test_sentences = sentences[train_size:]
test_y = y[train_size:]
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, maxlen=100, padding='post', truncating='post')
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(5000, 16, input_length=100),
#     tf.keras.layers.Bidirectional(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2)),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(9, activation='relu'),
#     tf.keras.layers.Dense(18, activation='relu'),

#     tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(3, activation='softmax')
])
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
model.summary()
model.fit(
    train_padded,
    train_y,
    epochs=15,
    validation_split=0.1
)
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, maxlen=100, padding='post', truncating='post')
model.evaluate(test_padded, test_y)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 16)           80000     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 9)                 153       
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 30        
Total params: 80,183
Trainable params: 80,183
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
436/436 [==============================] - 2s 3ms/step - loss: 0.9899 - accuracy: 0.5242 - val_loss: 0.9282 - val_accuracy: 0.5261
Epoch 2/15
436/436 [==============================] - 1s 3ms/step - loss: 0.8197 - accuracy: 0.6701 - val_lo

[0.39773622155189514, 0.8649625778198242]

In [448]:
#!g1.1
model.save('/home/jupyter/mnt/s3/main-data/models/model1.h5')

In [449]:
#!g1.1
train_size = int(len(sentences) * 0.8)
train_sentences = sentences[:train_size]
train_y = y[:train_size]
test_sentences = sentences[train_size:]
test_y = y[train_size:]
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, maxlen=100, padding='post', truncating='post')
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(5000, 16, input_length=100),
#     tf.keras.layers.Bidirectional(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2)),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(9, activation='tanh'),
#     tf.keras.layers.Dense(18, activation='relu'),

#     tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(3, activation='sigmoid')
])
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
model.summary()
model.fit(
    train_padded,
    train_y,
    epochs=15,
    validation_split=0.1
)
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, maxlen=100, padding='post', truncating='post')
model.evaluate(test_padded, test_y)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 16)           80000     
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 153       
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 30        
Total params: 80,183
Trainable params: 80,183
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
436/436 [==============================] - 2s 3ms/step - loss: 0.9763 - accuracy: 0.5256 - val_loss: 0.9058 - val_accuracy: 0.5707
Epoch 2/15
436/436 [==============================] - 1s 3ms/step - loss: 0.7768 - accuracy: 0.7098 - val_

[0.41964489221572876, 0.8665117621421814]

In [304]:
#!g1.1
model.save('/home/jupyter/mnt/s3/main-data/models/model2.h5')

In [305]:
#!g1.1
train_size = int(len(sentences) * 0.8)
train_sentences = sentences[:train_size]
train_y = y[:train_size]
test_sentences = sentences[train_size:]
test_y = y[train_size:]
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, maxlen=100, padding='post', truncating='post')
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(5000, 16, input_length=100),
#     tf.keras.layers.Bidirectional(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2)),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(9, activation='exponential'),
#     tf.keras.layers.Dense(18, activation='relu'),

#     tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(3, activation='softmax')
])
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
model.summary()
model.fit(
    train_padded,
    train_y,
    epochs=15,
    validation_split=0.1
)
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, maxlen=100, padding='post', truncating='post')
model.evaluate(test_padded, test_y)

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 100, 16)           80000     
_________________________________________________________________
global_average_pooling1d_20  (None, 16)                0         
_________________________________________________________________
dense_40 (Dense)             (None, 9)                 153       
_________________________________________________________________
dense_41 (Dense)             (None, 3)                 30        
Total params: 80,183
Trainable params: 80,183
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
436/436 [==============================] - 2s 3ms/step - loss: 1.0780 - accuracy: 0.4881 - val_loss: 0.9684 - val_accuracy: 0.5261
Epoch 2/15
436/436 [==============================] - 1s 3ms/step - loss: 0.9331 - accuracy: 0.5510 - val

[0.4364631175994873, 0.852827250957489]

In [306]:
#!g1.1
model.save('/home/jupyter/mnt/s3/main-data/models/model3.h5')

In [308]:
#!g1.1
train_size = int(len(sentences) * 0.8)
train_sentences = sentences[:train_size]
train_y = y[:train_size]
test_sentences = sentences[train_size:]
test_y = y[train_size:]
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, maxlen=100, padding='post', truncating='post')
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(5000, 16, input_length=100),
#     tf.keras.layers.Bidirectional(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2)),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(9, activation='relu'),
    tf.keras.layers.Dense(12, activation='relu'),

#     tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(3, activation='softmax')
])
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
model.summary()
model.fit(
    train_padded,
    train_y,
    epochs=15,
    validation_split=0.1
)
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, maxlen=100, padding='post', truncating='post')
model.evaluate(test_padded, test_y)

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_22 (Embedding)     (None, 100, 16)           80000     
_________________________________________________________________
global_average_pooling1d_22  (None, 16)                0         
_________________________________________________________________
dense_44 (Dense)             (None, 9)                 153       
_________________________________________________________________
dense_45 (Dense)             (None, 12)                120       
_________________________________________________________________
dense_46 (Dense)             (None, 3)                 39        
Total params: 80,312
Trainable params: 80,312
Non-trainable params: 0
_________________________________________________________________
Epoch 1/15
436/436 [==============================] - 2s 3ms/step - loss: 0.9844 - accuracy: 0.5243 - val_loss: 0.8

[0.43065235018730164, 0.8659953474998474]

In [309]:
#!g1.1
model.save('/home/jupyter/mnt/s3/main-data/models/model4.h5')

In [ ]:
#!g1.1
